In [104]:
import sys
import os
from elasticsearch import Elasticsearch
from zipfile import ZipFile
import re
import pandas as pd
import json
import requests
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import nltk
import pickle
import pytrec_eval
nltk.download('stopwords')
nltk.download('punkt')
from elasticsearch import NotFoundError

es = Elasticsearch()

[nltk_data] Downloading package stopwords to /home/katya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/katya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [105]:
input_dir = 'data/docs'
output_dir = 'b075k13'

In [106]:
def strip_punct(s):
    s = re.sub('[^A-Za-z0-9]', ' ', s)
    s = s.lower()
    return " ".join(s.split())

In [107]:
index = 'b075k13_index'

## Create new index, adjust BM25-parameters

In [108]:
request_body = {
    "settings" : {
	        "number_of_shards": 1,
	        "number_of_replicas": 0,
        "similarity": {
      "default": { 
        "type": "BM25",
        "b":0.75,
        "k1":1.3
      }
    }
	    }
	}

es.indices.create(index = index, body = request_body)

RequestError: RequestError(400, 'resource_already_exists_exception', 'index [b075k13_index/T4_pw98dTzyez3KoOsJBaw] already exists')

In [109]:
with open('bulk_data_json.txt') as f:
    bulk_data = json.load(f)

In [110]:
len(bulk_data)

10060

In [111]:
for x in bulk_data[::2]:
    x['index']['_index']=index
    #print(x)
print(bulk_data[0])

{'index': {'_index': 'b075k13_index', '_type': 'doc', '_id': 'clueweb12-1913wb-84-09751'}}


In [112]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [113]:
bulks = chunks(bulk_data, 100)

In [114]:
bulks

<generator object chunks at 0x7f41aa0bb2d0>

In [115]:
for x in bulks:
    res = es.bulk(index = index, body = x)


#es.search(body={"query": {"match_all": {}}}, index = 'example_index')
#es_indices.get_mapping(index = 'example_index')

/home/katya/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [116]:
res = es.get(index=index,doc_type='doc',id='clueweb12-1214wb-88-29751')

/home/katya/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document get requests is deprecated, use the /{index}/_doc/{id} endpoint instead.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [117]:
res

{'_index': 'b075k13_index',
 '_type': 'doc',
 '_id': 'clueweb12-1214wb-88-29751',
 '_version': 2,
 '_seq_no': 8663,
 '_primary_term': 1,
 'found': True,
 '_source': {'query': 'what and is and the and difference and between and sex and and and love',
  'title': '<em>Difference</em> <em>Between</em> <em>Love</em> <em>And</em> <em>Sex</em> - Student.com Articles',
  'title_lem': 'difference love sex student article',
  'num': '1',
  'uuid': 'b0054765-4007-52c9-a7c4-70c15891b4cc',
  'score': 2406.7341,
  'document': 'Difference Between Love And Sex\nYou are here:\nStudent.com Articles Student Health & Sexuality Sexual Health Difference Between Love And Sex\nRelated Articles\n<p>Search, Compare, and Apply for student loans at <a href="http://www.simpletuition.com?brnd=studentcom" title="Simpletuition.com" target="_blank">Simpletuition.com</a>.</p>\nPaying For College\nSex VS. Love Love and sex are NOT the same thing. Love is an emotion or a feeling. There is no one definition of love becaus

In [118]:
#es.indices.delete(index='b05k1_index')

In [119]:
es.cat.count(index, params={"format": "json"})

[{'epoch': '1613043268', 'timestamp': '11:34:28', 'count': '5017'}]

## Create different runs

In [120]:
f = open("data/topics_lemmatized.txt", 'rb')
queries_lem = pickle.load(f)
f.close()

In [121]:
def search(search_param, filename, q, ind):
    
    qid = []
    Q0 = []
    doc = []
    rank = []
    score = []
    tag = []

    num=1

    for i in q:
        for x in search_param['query']['bool']['should']:
            if type(i) == dict:
                for key in x['match']:
                    x['match'][key] = i['title']
            else:
                for key in x['match']:
                    x['match'][key] = i

    
        response = es.search(index=ind, body=search_param)
        r = 1
        for x in response['hits']['hits']:
            qid.append(num)
            Q0.append('Q0')
            doc.append(x['_id'])
            rank.append(r)
            score.append(x['_score'])
            tag.append('uh-t2-thor')
            r+=1
        num+=1
    
   
    qrels = {'qid': qid, 'Q0': Q0, 'doc': doc, 'rank':rank, 'score':score, 'tag':tag}

    df = pd.DataFrame(qrels)
    df.to_csv((output_dir+'/'+filename+'.txt'), sep = ' ', index = False, header = False)

In [122]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "title_lem": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_title', queries_lem, index)

In [123]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "lem": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_doc', queries_lem, index)

In [124]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "args": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_arg', queries_lem, index)

In [125]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_arg', queries_lem, index)

In [126]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "lem": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_doc', queries_lem, index)

In [127]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_doc_arg', queries_lem, index)

In [128]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "lem": ""
          }
        },
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_doc_arg', queries_lem, index)

## Add synonyms

In [129]:
def search_extended(search_param, filename, q, ind):
    
    qid = []
    Q0 = []
    doc = []
    rank = []
    score = []
    tag = []

    num=1
#['bool']['should']
    for idx, row in q.iterrows():
            
            #for key in x['match']:
                #x['match'][key] = row['query']
        for x in search_param['query']['bool']['should']:
            for key in x["match"]:
                #print(key)
                if "boost" in x['match'][key]:
                    x['match'][key]["query"] = row['query']
                else:
                    x['match'][key] = row['syn']
        #print(search_param)
        response = es.search(index=ind, body=search_param)
        r = 1
        for x in response['hits']['hits']:
            qid.append(num)
            Q0.append('Q0')
            doc.append(x['_id'])
            rank.append(r)
            score.append(x['_score'])
            tag.append('uh-t2-thor')
            r+=1
        num+=1
    
   
    qrels = {'qid': qid, 'Q0': Q0, 'doc': doc, 'rank':rank, 'score':score, 'tag':tag}

    df = pd.DataFrame(qrels)
    df.to_csv((output_dir+'/'+filename+'.txt'), sep = ' ', index = False, header = False)

In [130]:
df_syn = pd.read_csv('data/q_for_syn.tsv', sep = '\t')

In [131]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_syn', df_syn, index)

In [132]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_doc_syn', df_syn, index)

In [133]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_args_syn', df_syn, index)

In [134]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_arg_syn', df_syn, index)

In [135]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_doc_syn', df_syn, index)

In [136]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {""}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_doc_arg_syn', df_syn, index)

In [137]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"args": {""}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_doc_arg_syn', df_syn, index)

## Evaluate and compare results

In [138]:
def evaluate_run(run_path, qrel_path):
    run = pd.read_csv(run_path, sep = ' ', names = ['topic','Q0','id','rank','score','team'])

    qrel = pd.read_csv(qrel_path, sep = ' ', names = ['topic','Q0','id','relevance'])

    qrels = {}
    for i in range(1,51):
        qrels[str(i)] = {}
    
    for idx, row in qrel.iterrows():
        qrels[str(row['topic'])][row['id']] = row['relevance']
    
    runs = {}
    for i in range(1,51):
        runs[str(i)] = {}
    
    for idx, row in run.iterrows():
        runs[str(row['topic'])][row['id']] = row['score']

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrels,{'map_cut', 'ndcg_cut', 'recall', 'P'})

    res = evaluator.evaluate(runs)

    results = {}
    recall = {}
    pres = {}
    for key in res:
        results[key] = res[key]['ndcg_cut_10']
        recall[key] = res[key]['recall_10']
        pres[key] = res[key]['P_10']
        #print(res[key])
    
    filename = re.split('\.', sys.argv[1])
    with open((filename[0]+'_evaluated.json'), 'w') as f:
        json.dump(results, f)
        
    s = sum(results.values())/50
    r = sum(recall.values())/50
    p = sum(pres.values())/50
    if r+p != 0:
        f1 = (2*(r*p)/(r+p))
    else: 
        f1 = 0
    '''
    print("Average ndcg_cut_10: ", s)
    print("Average recall_10: ", r)
    print("Av. precision: ", p)
    print("F1-score: ",f1)
    '''    
    return s, r, p, f1


In [139]:
run = []
ndcg = []
rec = []
prec = []
f1s = []

for filename in os.listdir(output_dir):
    if filename.endswith(".txt"):
        #print(filename)
        s, r, p, f1 = evaluate_run((output_dir+"/{}").format(filename), 'touche2020-task2-relevance-withbaseline(1).qrels')
        run.append(filename)
        ndcg.append(s)
        rec.append(r)
        prec.append(p)
        f1s.append(f1)

In [140]:
df = pd.DataFrame({'run':run,'ndcg_cut10':ndcg, 'recall_10':rec, 'precision':prec, 'f1-score':f1s})

### All results

In [141]:
df.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
10,run_title_doc_syn.txt,0.437558,0.280096,0.428,0.338601
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
1,run_title_arg_syn.txt,0.379502,0.246911,0.366,0.294886
0,run_title_doc_arg.txt,0.377465,0.234616,0.356,0.282834
2,run_title.txt,0.373800,0.234699,0.366,0.286000
11,run_title_doc_arg_syn.txt,0.371223,0.231843,0.354,0.280186
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
7,run_doc.txt,0.318739,0.193800,0.292,0.232975
4,run_doc_syn.txt,0.317996,0.194886,0.296,0.235029


In [142]:
df.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.437558,0.280096,0.428,0.338601
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
1,run_title_arg_syn.txt,0.379502,0.246911,0.366,0.294886
2,run_title.txt,0.373800,0.234699,0.366,0.286000
0,run_title_doc_arg.txt,0.377465,0.234616,0.356,0.282834
11,run_title_doc_arg_syn.txt,0.371223,0.231843,0.354,0.280186
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
4,run_doc_syn.txt,0.317996,0.194886,0.296,0.235029
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


In [143]:
df.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.437558,0.280096,0.428,0.338601
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
1,run_title_arg_syn.txt,0.379502,0.246911,0.366,0.294886
2,run_title.txt,0.373800,0.234699,0.366,0.286000
0,run_title_doc_arg.txt,0.377465,0.234616,0.356,0.282834
11,run_title_doc_arg_syn.txt,0.371223,0.231843,0.354,0.280186
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
4,run_doc_syn.txt,0.317996,0.194886,0.296,0.235029
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


In [144]:
df.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.437558,0.280096,0.428,0.338601
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
1,run_title_arg_syn.txt,0.379502,0.246911,0.366,0.294886
2,run_title.txt,0.373800,0.234699,0.366,0.286000
0,run_title_doc_arg.txt,0.377465,0.234616,0.356,0.282834
11,run_title_doc_arg_syn.txt,0.371223,0.231843,0.354,0.280186
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
4,run_doc_syn.txt,0.317996,0.194886,0.296,0.235029
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


### Results without doc

In [145]:
df_p = df[~df.run.str.contains("doc")]

In [146]:
df_p.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
1,run_title_arg_syn.txt,0.379502,0.246911,0.366,0.294886
2,run_title.txt,0.373800,0.234699,0.366,0.286000
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
8,run_args_syn.txt,0.267730,0.179784,0.260,0.212576
9,run_arg.txt,0.265178,0.177569,0.260,0.211020


In [147]:
df_p.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
1,run_title_arg_syn.txt,0.379502,0.246911,0.366,0.294886
2,run_title.txt,0.373800,0.234699,0.366,0.286000
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
8,run_args_syn.txt,0.267730,0.179784,0.260,0.212576
9,run_arg.txt,0.265178,0.177569,0.260,0.211020


In [148]:
df_p.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
1,run_title_arg_syn.txt,0.379502,0.246911,0.366,0.294886
2,run_title.txt,0.373800,0.234699,0.366,0.286000
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
8,run_args_syn.txt,0.267730,0.179784,0.260,0.212576
9,run_arg.txt,0.265178,0.177569,0.260,0.211020


In [149]:
df_p.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
1,run_title_arg_syn.txt,0.379502,0.246911,0.366,0.294886
2,run_title.txt,0.373800,0.234699,0.366,0.286000
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
8,run_args_syn.txt,0.267730,0.179784,0.260,0.212576
9,run_arg.txt,0.265178,0.177569,0.260,0.211020


### Results for pipeline

In [150]:
df_p = df[~df.run.str.contains("arg")]
df_p = df_p[~df_p.run.str.contains("syn")]

In [151]:
df_p.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
2,run_title.txt,0.373800,0.234699,0.366,0.286000
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


In [152]:
df_p.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
2,run_title.txt,0.373800,0.234699,0.366,0.286000
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


In [153]:
df_p.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
2,run_title.txt,0.373800,0.234699,0.366,0.286000
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


In [154]:
df_p.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
2,run_title.txt,0.373800,0.234699,0.366,0.286000
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


### Results for synonyms

In [155]:
df_syn = df[~df.run.str.contains("arg")]

In [156]:
df_syn.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
10,run_title_doc_syn.txt,0.437558,0.280096,0.428,0.338601
2,run_title.txt,0.373800,0.234699,0.366,0.286000
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
7,run_doc.txt,0.318739,0.193800,0.292,0.232975
4,run_doc_syn.txt,0.317996,0.194886,0.296,0.235029


In [157]:
df_syn.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.437558,0.280096,0.428,0.338601
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
2,run_title.txt,0.373800,0.234699,0.366,0.286000
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
4,run_doc_syn.txt,0.317996,0.194886,0.296,0.235029
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


In [158]:
df_syn.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.437558,0.280096,0.428,0.338601
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
2,run_title.txt,0.373800,0.234699,0.366,0.286000
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
4,run_doc_syn.txt,0.317996,0.194886,0.296,0.235029
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


In [159]:
df_syn.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.437558,0.280096,0.428,0.338601
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
2,run_title.txt,0.373800,0.234699,0.366,0.286000
13,run_title_syn.txt,0.350099,0.219121,0.340,0.266494
4,run_doc_syn.txt,0.317996,0.194886,0.296,0.235029
7,run_doc.txt,0.318739,0.193800,0.292,0.232975


### Results for arguments

In [160]:
df_arg = df[~df.run.str.contains("syn")]

In [161]:
df_arg.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
0,run_title_doc_arg.txt,0.377465,0.234616,0.356,0.282834
2,run_title.txt,0.373800,0.234699,0.366,0.286000
7,run_doc.txt,0.318739,0.193800,0.292,0.232975
6,run_doc_arg.txt,0.284590,0.187975,0.276,0.223637
9,run_arg.txt,0.265178,0.177569,0.260,0.211020


In [162]:
df_arg.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
2,run_title.txt,0.373800,0.234699,0.366,0.286000
0,run_title_doc_arg.txt,0.377465,0.234616,0.356,0.282834
7,run_doc.txt,0.318739,0.193800,0.292,0.232975
6,run_doc_arg.txt,0.284590,0.187975,0.276,0.223637
9,run_arg.txt,0.265178,0.177569,0.260,0.211020


In [163]:
df_arg.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
2,run_title.txt,0.373800,0.234699,0.366,0.286000
0,run_title_doc_arg.txt,0.377465,0.234616,0.356,0.282834
7,run_doc.txt,0.318739,0.193800,0.292,0.232975
6,run_doc_arg.txt,0.284590,0.187975,0.276,0.223637
9,run_arg.txt,0.265178,0.177569,0.260,0.211020


In [164]:
df_arg.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.437613,0.270779,0.422,0.329885
12,run_title_arg.txt,0.386540,0.249770,0.372,0.298871
2,run_title.txt,0.373800,0.234699,0.366,0.286000
0,run_title_doc_arg.txt,0.377465,0.234616,0.356,0.282834
7,run_doc.txt,0.318739,0.193800,0.292,0.232975
6,run_doc_arg.txt,0.284590,0.187975,0.276,0.223637
9,run_arg.txt,0.265178,0.177569,0.260,0.211020
